In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('datasets/embedded_dataset.csv')
pd.set_option('display.max_rows', None)
df.head()

,customer_id,timestamp,action
0,0,2023-01-24 12:00:00,app_login
1,0,2023-01-24 12:05:00,app_login
2,0,2023-01-24 12:06:00,gold_loan_renew
3,0,2023-07-19 15:09:00,gold_loan_renew
4,0,2023-07-19 15:20:00,gold_loan_renew


In [12]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

all_actions = ['app_login', 'gold_loan_renew', 'transaction_imps',
           'priority_banking_request', 'fd_open_1', 'fd_open_2', 'rd_opening',
           'transaction_own_acc', 'fd_open_5', 'billpay_recharge',
           'kyc_update_DrivingLicence', 'FOR_add_beneficiary', 'FOR_send_money',
           'fsld_open', 'kyc_update_Aadhaar/Address', 'kyc_update_PAN',
           'kyc_update_VoterID', 'fast_tag', 'mail_id_update', 'transaction_neft',
           'fd_open_8', 'fd_open_12', 'fd_open_4', 'fd_open_3', 'fd_open_10',
           'apply_apy', 'fd_open_7', 'fd_open_9', 'fd_open_6', 'fd_open_13',
           'fd_open_14', 'fd_open_11', 'fd_open_16', 'fd_open_15']

sequences = []
next_actions = []
sequence_length = 5

df.sort_values(by=['customer_id', 'timestamp'], inplace=True)

# Tokenize the actions
action_tokenizer = Tokenizer()
action_tokenizer.fit_on_texts(all_actions)

# Creating sequences and next_actions
for customer_id, group in df.groupby('customer_id'):
    customer_actions = group['action'].tolist()
    for i in range(len(customer_actions) - sequence_length):
        sequence = customer_actions[i:i + sequence_length]
        next_action = customer_actions[i + sequence_length]
        sequences.append(sequence)
        next_actions.append(next_action)

# Converting sequences and next_actions to numerical representation
X = action_tokenizer.texts_to_sequences(sequences)
# Pad all of the sequences to the same length
X = pad_sequences(X, maxlen=sequence_length, padding='pre')

y = action_tokenizer.texts_to_sequences(next_actions)
y = pad_sequences(y, maxlen=sequence_length, padding='pre')
y = to_categorical(y, num_classes=len(action_tokenizer.word_index) + 1)

X = np.array(X)
y = np.array(y)

y_padded = pad_sequences(y, maxlen=sequence_length, padding='post')

# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(action_tokenizer.word_index) + 1, output_dim=1, input_length=sequence_length))
model.add(LSTM(units=100, return_sequences=True))
model.add(LSTM(units=100))
model.add(Dense(units=len(action_tokenizer.word_index) + 1, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

new_sequence = action_tokenizer.texts_to_sequences(["login", "view_balance", "transfer_money", "view_balance", "logout"])
# Pad the new sequence to the same length as the other sequences
new_sequence = pad_sequences([new_sequence], maxlen=sequence_length, padding='pre')

predicted_probabilities = model.predict(new_sequence)
predicted_action_index = np.argmax(predicted_probabilities)
predicted_action = action_tokenizer.index_word[predicted_action_index]

# Print the predicted action
print("Predicted action:", predicted_action)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\alosh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\alosh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\alosh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\alosh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\alosh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\alosh\anaconda3\envs\tf\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = 